## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To upload the Resume in PDF and convert it to text and ask questions based on the resume using gradio
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simple, brute-force type of RAG..

# Installing the necessary libraries

In [1]:
!pip install --q pdfplumber

# Importing all the neccessary libraries

In [6]:
import os
import glob
import pdfplumber
from dotenv import load_dotenv
from pathlib import Path
import gradio as gr

In [26]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
BASE_DIR = Path().resolve()
pdf_dir  = BASE_DIR / "resumes_pdf"
text_dir = BASE_DIR / "resumes_txt"

print("pdf_dir  is a", type(pdf_dir))
print("text_dir is a", type(text_dir))

pdf_dir  is a <class 'pathlib.WindowsPath'>
text_dir is a <class 'pathlib.WindowsPath'>


In [12]:
pdf_dir

WindowsPath('C:/Users/DELL/Documents/Projects/llm_engineering/week5/resumes_pdf')

In [13]:
pdf_files = sorted(pdf_dir.glob("*.pdf"))
print(f"Found {len(pdf_files)} PDFs:")
for p in pdf_files:
    print(" ", p.name)

Found 2 PDFs:
  Profile (1).pdf
  Rishi Kora Resume.pdf


In [14]:
for pdf_path in pdf_files:
    txt_path = text_dir / f"{pdf_path.stem}.txt"
    with pdfplumber.open(pdf_path) as pdf, \
         open(txt_path, "w", encoding="utf-8") as out_f:
        for i, page in enumerate(pdf.pages, start=1):
            out_f.write(f"--- Page {i} ---\n")
            out_f.write(page.extract_text() or "[No extractable text]")
            out_f.write("\n\n")
    print(f"Converted {pdf_path.name} → {txt_path.name}")
print("All done!")

Converted Profile (1).pdf → Profile (1).txt
Converted Rishi Kora Resume.pdf → Rishi Kora Resume.txt
All done!


# Choosing a model from OpenAI which is low and creating a vector database

In [15]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

# Load environment variables in a file called `.env`

In [16]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# This code loads `.md` files from subfolders in resumes_txt/, tags each with its folder name as metadata, and splits them into smaller text chunks. It helps organize and prepare data for use in RAG-based AI applications.

In [21]:
folders = glob.glob("resumes_txt/*") 

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {"encoding": "utf-8"}

documents = []
entries = glob.glob("resumes_txt/*")

for entry in entries:
    doc_type = os.path.splitext(os.path.basename(entry))[0]

    if os.path.isdir(entry):
        # Load all .md files from the folder
        loader = DirectoryLoader(
            path=entry,
            glob="**/*.md",
            loader_cls=TextLoader,
            loader_kwargs=text_loader_kwargs
        )
        folder_docs = loader.load()
    elif os.path.isfile(entry) and entry.endswith(".txt"):
        # Load a single .txt file
        loader = TextLoader(entry, **text_loader_kwargs)
        folder_docs = loader.load()
    else:
        continue  # skip unsupported file types

    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# Summary
print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Created a chunk of size 1878, which is longer than the specified 1000


Total number of chunks: 3
Document types found: {'Rishi Kora Resume', 'Profile (1)'}


In [22]:
embeddings = OpenAIEmbeddings()
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 3 documents


In [23]:
collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 3 vectors with 1,536 dimensions in the vector store


In [28]:
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15740\2669631352.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [30]:
query = "Who is Rishi Kora"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


Rishi Kora is an aspiring LLM Engineer who recently graduated with a Master's degree in Data Science from the University of Essex. He specializes in LLM engineering and has experience building end-to-end transformer pipelines, implementing RAG systems, and deploying LLMs. He holds a UK Post-Study Work Visa valid through 2026 and is actively seeking entry-level LLM Engineering roles in the UK. Rishi is proficient in several programming languages and tools, including Python, PyTorch, Hugging Face, and AWS SageMaker.


In [31]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [32]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [34]:
view = gr.ChatInterface(chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7447609f659bf9960a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
